In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb209.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb157.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/dwnpl05139.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/dnypl05025.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb191.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/dnypl05005.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb62.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb438.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb479.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/ddfgf10167.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/dhzgf10146.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb66.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb314.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/dnypl05028.png
/kaggle/input/landslidemask/withMaskes/Non-Landslides/fyb192.png
/ka

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [3]:
# Directories
landslide_dir = r"/kaggle/input/landslidemask/withMaskes/Landslide"
landslide_mask_dir = r"/kaggle/input/landslidemask/withMaskes/LandslideMask"
non_landslide_dir = r"/kaggle/input/landslidemask/withMaskes/Non-Landslides"

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 8

In [4]:
def preprocess_images_and_masks():
    images = []
    masks = []

    # Process landslide images and masks
    for img_name in os.listdir(landslide_dir):
        img_path = os.path.join(landslide_dir, img_name)
        mask_path = os.path.join(landslide_mask_dir, img_name.replace('.jpg', '_mask.png'))

        if os.path.exists(mask_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, IMG_SIZE) / 255.0

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, IMG_SIZE) / 255.0
            mask = np.expand_dims(mask, axis=-1)  # Add channel dimension

            images.append(img)
            masks.append(mask)

    # Process non-landslide images
    for img_name in os.listdir(non_landslide_dir):
        img_path = os.path.join(non_landslide_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, IMG_SIZE) / 255.0

        mask = np.zeros((IMG_SIZE[0], IMG_SIZE[1], 1))  # Mask filled with zeros and channel dimension

        images.append(img)
        masks.append(mask)

    # Convert to numpy arrays and add batch dimension
    images = np.array(images, dtype=np.float32)
    masks = np.array(masks, dtype=np.float32)

    return images, masks


In [5]:
# Load data
images, masks = preprocess_images_and_masks()

# Train-validation split
split = int(0.8 * len(images))
train_images, val_images = images[:split], images[split:]
train_masks, val_masks = masks[:split], masks[split:]

# Data augmentation
data_gen_args = dict(horizontal_flip=True, rotation_range=15)
image_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

#

train_image_gen = image_data_gen.flow(train_images, batch_size=BATCH_SIZE, seed=42)
train_mask_gen = mask_data_gen.flow(train_masks, batch_size=BATCH_SIZE, seed=42)

# Zip the generators to synchronize them
train_gen = zip(train_image_gen, train_mask_gen)

val_image_gen = image_data_gen.flow(val_images, batch_size=BATCH_SIZE, seed=42)
val_mask_gen = mask_data_gen.flow(val_masks, batch_size=BATCH_SIZE, seed=42)

val_gen = zip(val_image_gen, val_mask_gen)

In [6]:
def data_generator(image_gen, mask_gen):
    while True:
        for img, mask in zip(image_gen, mask_gen):
            yield img, mask

In [7]:
train_gen = data_generator(train_image_gen, train_mask_gen)
val_gen = data_generator(val_image_gen, val_mask_gen)

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def transformer_encoder(inputs, num_heads, key_dim, ff_dim, dropout=0.1):
    """
    A simple implementation of a Transformer encoder block.
    Parameters:
    - inputs: Input tensor to the Transformer block.
    - num_heads: Number of attention heads.
    - key_dim: Dimension of the keys in the attention mechanism.
    - ff_dim: Dimension of the feed-forward network.
    - dropout: Dropout rate.
    Returns:
    - Output tensor of the Transformer encoder block.
    """
    # Multi-head self-attention
    attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(inputs, inputs)
    attention_output = layers.Dropout(dropout)(attention_output)
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output + inputs)

    # Feed-forward network
    ff_output = layers.Dense(ff_dim, activation="relu")(attention_output)
    ff_output = layers.Dense(inputs.shape[-1])(ff_output)
    ff_output = layers.Dropout(dropout)(ff_output)
    output = layers.LayerNormalization(epsilon=1e-6)(ff_output + attention_output)

    return output




In [15]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model

In [22]:
def build_vit_unet(input_size=(224, 224, 3)):
    # Input layer
    inputs = layers.Input(shape=input_size)

    # Patch embedding (simulating a Vision Transformer behavior)
    patch_size = 16
    num_patches = (input_size[0] // patch_size) * (input_size[1] // patch_size)
    x = layers.Conv2D(64, (patch_size, patch_size), strides=(patch_size, patch_size))(inputs)
    x = layers.Reshape((num_patches, -1))(x)

    # Transformer encoder block
    x = transformer_encoder(x, num_heads=4, key_dim=64, ff_dim=128, dropout=0.1)

    # Reshape back to spatial dimensions
    x = layers.Reshape((input_size[0] // patch_size, input_size[1] // patch_size, 64))(x)

    # Decoder (similar to UNet)
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x)
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="relu")(x)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding="same", activation="relu")(x)
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)

    # **Additional Upsampling Layer** to match (224, 224)
    x = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding="same", activation="relu")(x)
    x = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(x)

    # Final output layer
    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid", name="output_layer")(x)

    # Build the model
    model = Model(inputs, outputs)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    return model


In [23]:
# Build the model
model = build_vit_unet()

# Print the model summary
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 14, 14,    │     49,216 │ input_layer_7[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_11          │ (None, 196, 64)   │          0 │ conv2d_27[0][0]   │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 196, 64)   │     66,368 │ reshape_11[0][0], │
│ (MultiHeadAttentio… │                   │            │ reshape_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 196, 64)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 196, 64)   │          0 │ dropout_16[0][0], │
│                     │                   │            │ reshape_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 196, 64)   │        128 │ add_10[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 196, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 196, 64)   │      8,256 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 196, 64)   │          0 │ dense_11[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_11 (Add)        │ (None, 196, 64)   │          0 │ dropout_17[0][0], │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 196, 64)   │        128 │ add_11[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_12          │ (None, 14, 14,    │          0 │ layer_normalizat… │
│ (Reshape)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_21 │ (None, 28, 28,    │     73,856 │ reshape_12[0][0]  │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_28 (Conv2D)  │ (None, 28, 28,    │    147,584 │ conv2d_transpose… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_22 │ (None, 56, 56,    │     73,792 │ conv2d_28[0][0]   │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_29 (Conv2D)  │ (None, 56, 56,    │     36,928 │ conv2d_transpose… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 499,249 (1.90 MB)

 Trainable params: 499,249 (1.90 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    steps_per_epoch=len(train_images) // BATCH_SIZE,
    validation_steps=len(val_images) // BATCH_SIZE
)

Epoch 1/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 28s 121ms/step - accuracy: 0.9336 - loss: 0.2904 - val_accuracy: 1.0000 - val_loss: 0.1280
Epoch 2/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 18s 112ms/step - accuracy: 0.9327 - loss: 0.1698 - val_accuracy: 1.0000 - val_loss: 0.0268
Epoch 3/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 17s 109ms/step - accuracy: 0.9309 - loss: 0.1765 - val_accuracy: 1.0000 - val_loss: 0.0214
Epoch 4/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 16s 104ms/step - accuracy: 0.9328 - loss: 0.1581 - val_accuracy: 1.0000 - val_loss: 0.0588
Epoch 5/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 14s 89ms/step - accuracy: 0.9350 - loss: 0.1562 - val_accuracy: 1.0000 - val_loss: 0.0477
Epoch 6/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 15s 94ms/step - accuracy: 0.9361 - loss: 0.1521 - val_accuracy: 1.0000 - val_loss: 0.1028
Epoch 7/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 15s 95ms/step - accuracy: 0.9344 - loss: 0.1659 - val_accuracy: 1.0000 - val_loss: 0.0273
Epoch 8/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 15s 94ms/step - accuracy: 0.9331 - loss: 0.154